In [ ]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

In [ ]:

%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config_loader import config
from src.shop import generate_product_images, generate_shopify_csv
from src.zip import create_m4a_zip_collections
print(config.TARGET_LANGUAGE_NAME)

# Load product configurations from JSON file
import json
with open('../src/product_configs.json', 'r') as f:
    PRODUCT_CONFIG = json.load(f)

# Set the current collection
COLLECTION = "WarmUp150"
current_product_config = PRODUCT_CONFIG[COLLECTION]

print(f"Loaded product config for {COLLECTION}")
print(f"Individual indices: {current_product_config['individual']['indices']}")
print(f"Bundle ranges: {current_product_config['bundle']['ranges']}")
print(f"Complete pack price: £{current_product_config['complete']['price']}")


In [ ]:

# Option 1: Use the automated script to run all Shopify material generation steps
print("=== Automated Pipeline Option ===")
print(f"To run all Shopify material generation steps for {COLLECTION}, use:")
print(f"!python ../src/process_collection_to_new_language.py {COLLECTION} --start-from zip")
print(f"")
print("Or to run individual steps:")
print(f"!python ../src/process_collection_to_new_language.py {COLLECTION} --start-from zip --skip-images --skip-csv")
print("")
print("=== Manual Step-by-Step Option ===")
print("Run the cells below to execute each step individually:")

In [ ]:
# Run the automated script for all Shopify material generation
# Uncomment the line below to run the complete pipeline:
# !python ../src/process_collection_to_new_language.py {COLLECTION} --start-from zip

print(f"To run the complete pipeline, execute:")
print(f"!python ../src/process_collection_to_new_language.py {COLLECTION} --start-from zip")


In [ ]:
# Manual Step 1: Create M4A Zip Collections
print("=== Step 1: Creating M4A Zip Collections ===")
zips = create_m4a_zip_collections(
    product_config=current_product_config,
    collection=COLLECTION,
    bucket_name=config.GCS_PRIVATE_BUCKET,
    output_base_dir="../outputs/gcs",
    use_local_files=True
)

print(f"✅ Created {len(zips)} zip files:")
for zip_type, zip_path in zips.items():
    print(f"  {zip_type}: {zip_path}")

In [ ]:
# Manual Step 2: Generate Product Images
print("=== Step 2: Generating Product Images ===")
generated_images = generate_product_images(
    product_config=current_product_config,
    collection=COLLECTION, 
    generate_individual=True
)

print(f"✅ Generated {len(generated_images)} product images:")
for product_type, uri in generated_images.items():
    print(f"  {product_type}: {uri}")


In [ ]:

# Manual Step 3: Generate Shopify CSV
print("=== Step 3: Generating Shopify CSV ===")

csv_path = generate_shopify_csv(
    product_config=current_product_config,
    collection=COLLECTION,
    free_individual_count=1
)

print(f"✅ Shopify CSV generated at: {csv_path}")



In [27]:

print(f"Current language: {config.TARGET_LANGUAGE_NAME}")
print(f"Current language code: {config.TARGET_LANGUAGE_CODE}")

In [ ]:
config.TARGET_LANGUAGE_NAME